Importar data

In [1]:
import pandas as pd
noSolutionDF = pd.read_csv('output_nosol.csv')
solutionDF = pd.read_csv('output_sol.csv')

Etiquetas

In [6]:
import numpy as np
noSolutionCount = noSolutionDF.shape[0]
solutionCount = solutionDF.shape[0]

print(noSolutionCount)
print(solutionCount)

noSolutionLabel = [0] * noSolutionCount  * 6
solutionLabel = [1] * solutionCount * 6
labels = noSolutionLabel + solutionLabel


Y = np.array(labels)

1339
1331
(2,)


X tableros

In [3]:
def Rotate_90(tablero):
    return np.rot90(tablero, k=1)

def Rotate_180(tablero):
    return np.rot90(tablero, k=2)

def Rotate_270(tablero):
    return np.rot90(tablero, k=3)

def FlipX(tablero):
    return np.fliplr(tablero)

def FlipY(tablero):
    return np.flipud(tablero)

boards = []

for indice, fila in noSolutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)
  
    
print("El tamaño de la lista `boards` es:", len(boards))
for indice, fila in solutionDF.iterrows():
    tableros_en_celda = fila['Board'].replace("\r", "").split('\n')
    tableros_en_celda = list(filter(lambda x: x != '', tableros_en_celda))
    tablero_matriz = [list(fila) for fila in tableros_en_celda]
    boards.append(tablero_matriz)

    # Data augmentation
aug_boards = []
for board in boards:
    board_np = np.array(board)
    aug_boards.append(board_np)  
    aug_boards.append(Rotate_90(board_np))  
    aug_boards.append(Rotate_180(board_np)) 
    aug_boards.append(Rotate_270(board_np))  
    aug_boards.append(FlipX(board_np))  
    aug_boards.append(FlipY(board_np))  

print("El tamaño de la lista `boards` es:", len(aug_boards))
with np.printoptions(threshold=np.inf):
    print(aug_boards[0])


# Imprie
'''for i, lista in enumerate(boards):
    print(f"Lista de tableros {i+1}:")
    for tablero in lista:
        print(tablero)
    print()  # Salto de línea entre listas de tablero'''

El tamaño de la lista `boards` es: 1339
El tamaño de la lista `boards` es: 16020
[['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']
 ['#' '#' '#' ' ' ' ' ' ' '#' '$' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' ' ' '$' ' ' '$' ' ' ' ' ' ' '$' ' ' '#' '$' ' ' '#']
 ['#' '#' '#' ' ' '#' '$' '#' ' ' '#' '#' '#' ' ' ' ' '$' ' ' '#']
 ['#' '#' '#' ' ' '#' ' ' '.' '.' '.' '.' '.' '#' '#' '$' '#' '#']
 ['#' '#' '#' ' ' ' ' '.' '.' '.' '+' '.' '.' ' ' ' ' '$' ' ' '#']
 ['#' '#' '#' '#' '$' '#' ' ' '.' '.' '.' '.' '#' ' ' '#' ' ' '#']
 ['#' ' ' ' ' ' ' ' ' '#' '#' '#' '#' ' ' '#' '$' '$' '$' ' ' '#']
 ['#' ' ' '$' '#' ' ' ' ' ' ' ' ' ' ' '$' ' ' '$' ' ' '#' ' ' '#']
 ['#' ' ' '$' ' ' '.' '.' '#' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '#']
 ['#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#' '#']]


'for i, lista in enumerate(boards):\n    print(f"Lista de tableros {i+1}:")\n    for tablero in lista:\n        print(tablero)\n    print()  # Salto de línea entre listas de tablero'

Tableros a tensor

In [4]:
import numpy as np

def tablero_a_tensor(tablero):
    alto = len(tablero)
    ancho = len(tablero[0])
    tensor = np.zeros((alto, ancho, 7))

    for i in range(alto):
        for j in range(ancho):
            elemento = tablero[i][j]
            if elemento == '#':
                tensor[i, j, 0] = 1
            elif elemento == '$':
                tensor[i, j, 1] = 1
            elif elemento == '.':
                tensor[i, j, 2] = 1
            elif elemento == '@':
                tensor[i, j, 3] = 1
            elif elemento == '*':
                tensor[i, j, 4] = 1
            elif elemento == '+':
                tensor[i, j, 5] = 1
            elif elemento == ' ':
                tensor[i, j, 6] = 1
                
    return tensor


X = [tablero_a_tensor(tablero) for tablero in aug_boards]
print("Tensor")
with np.printoptions(threshold=np.inf):
    print(X[0])


max_height = max(tensor.shape[0] for tensor in X)
max_width = max(tensor.shape[1] for tensor in X)


X_padded = np.ones((len(X), max_height, max_width, 7))

for idx, tensor in enumerate(X):
    height = tensor.shape[0]
    width = tensor.shape[1]
    X_padded[idx, :height, :width, :] = tensor

with np.printoptions(threshold=np.inf):
    print(X_padded[0])


'''print("Dimensiones de X_padded:", X_padded.shape)

# Verificar las dimensiones de X y y
#print("Dimensiones de X:", X.shape)
print("Dimensiones de y:", Y.shape)'''

Tensor
[[[1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0.

'print("Dimensiones de X_padded:", X_padded.shape)\n\n# Verificar las dimensiones de X y y\n#print("Dimensiones de X:", X.shape)\nprint("Dimensiones de y:", Y.shape)'

FSN

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, MaxPooling2D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=23)
print(len(X_train), len(Y_train), len(X_test), len(Y_test))


input_shape = (None, None, 7)  
inputs = Input(shape=input_shape)

# Arq
x = Conv2D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
x = Conv2D(filters=32, kernel_size=2, activation='relu', padding='same')(x)
x = Conv2D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)


model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
#model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_data=(X_test, Y_test))

# History
history = model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_data=(X_test, Y_test))

# Graficar
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()


12816 12816 3204 3204


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 7)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, None, None, 32) │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, None, None, 32) │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, None, None, 64) │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, None, None, 64) │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, None, None, 1)  │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,897 (112.88 KB)

 Trainable params: 28,897 (112.88 KB)

 Non-trainable params: 0 (0.00 B)

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 15, 16, 8, 8, 17, 10, 10, 7, 8, 9, 8, 8, 12, 6, 10, 29, 9, 10, 13, 10, 8, 10, 8, 8, 11, 12, 8, 9, 7, 9, 22, 12, 11, 16, 9, 11, 21, 11, 8, 8, 11, 16, 11, 11, 10, 9, 10, 13, 13, 9, 8, 9, 15, 7, 8, 9, 9, 16, 9, 10, 7, 8, 11, 19, 9, 11, 10, 11, 11, 10, 10, 8, 14, 27, 10, 17, 11, 8, 8, 15, 9, 9, 9, 8, 8, 11, 8, 12, 10, 12, 21, 19, 9, 9, 16, 9, 10, 8, 9, 10, 7, 10, 13, 9, 8, 7, 15, 8, 8, 20, 8, 19, 8, 16, 11, 16, 16, 18, 14, 22, 11, 11, 12, 10, 11, 9, 11, 9, 11, 10, 10, 8, 21, 10, 10, 7, 11, 9, 10, 8, 8, 8, 25, 13, 9, 9, 15, 8, 13, 14, 6, 8, 9, 15, 18, 9, 9, 10, 8, 8, 9, 9, 9, 9, 12, 13, 9, 7, 11, 7, 8, 12, 17, 8, 9, 13, 11, 7, 9, 9, 8, 17, 21, 9, 11, 10, 17, 7, 17, 10, 10, 15, 13, 7, 9, 8, 8, 9, 8, 8, 8, 15, 17, 7, 12, 13, 9, 8, 8, 10, 9, 9, 11, 7, 8, 9, 9, 9, 11, 9, 12, 9, 16, 15, 7, 9, 11, 10, 14, 9, 9, 11, 10, 8, 9, 8, 9, 17, 17, 14, 11, 13, 11, 11, 7, 12, 10, 13, 9, 9, 8, 10, 10